<a href="https://colab.research.google.com/github/ferdinandrafols/IA_LLMs/blob/main/gsi073_aula0_seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparação dos dados

Esta tarefa é inverter sequências de caracteres. Exemplo: **aabcd** em **dcbaa**.


In [ ]:
import torch
import torch.nn as nn
import random

# ===== 1. Dicionário e funções básicas =====
chars = list("abcde ")
vocab = {ch: i for i, ch in enumerate(chars)}  # Cada letra recebe um número
inv_vocab = {i: ch for ch, i in vocab.items()} # Tabela de decodificação
vocab_size = len(vocab)

def encode(s):  # Codifica letras em números
    return torch.tensor([vocab[c] for c in s], dtype=torch.long)

def decode(t):  # Decodifica números em letras
    return ''.join(inv_vocab[int(x)] for x in t)

def random_seq(n=5):  # Cria novas sequências aleatórias
    return ''.join(random.choice(chars[:-1]) for _ in range(n))

# ===== 2. Gerar dados =====
pairs = [(encode(s), encode(s[::-1])) for s in [random_seq() for _ in range(50000)]]
max_len = max(len(x) for x, _ in pairs)

def pad(x):  # Preenche com o caractere espaço (' ')
    return torch.cat([x, torch.tensor([vocab[' ']] * (max_len - len(x)))], dim=0)

inputs = torch.stack([pad(x) for x, _ in pairs])
targets = torch.stack([pad(y) for _, y in pairs])

train_ds = torch.utils.data.TensorDataset(inputs, targets)
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=128, shuffle=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# ===== 3. Prints para inspecionar =====
print(f"Vocabulário: {vocab}")
print(f"Tamanho do vocabulário: {vocab_size}")
print(f"Tamanho máximo das sequências (max_len): {max_len}")

# Mostrar 3 exemplos codificados/decodificados
for i in range(3):
    s = random_seq()
    encoded = encode(s)
    decoded = decode(encoded)
    reversed_decoded = decode(encoded.flip(0))
    print(f"\nExemplo {i+1}:")
    print(f"  Original: {s}")
    print(f"  Codificado: {encoded.tolist()}")
    print(f"  Decodificado: {decoded}")
    print(f"  Reverso (target esperado): {reversed_decoded}")

# Mostrar formas (shapes) dos tensores de entrada e saída
print("\nShapes:")
print(f"  inputs:  {inputs.shape}")
print(f"  targets: {targets.shape}")

# Mostrar o primeiro batch do DataLoader
for xb, yb in train_dl:
    print("\nPrimeiro batch de treino:")
    print("  Entradas (xb):", xb.shape)
    print("  Alvos (yb):", yb.shape)
    print("  Exemplo de entrada decodificada:", decode(xb[0]))
    print("  Exemplo de alvo decodificado:", decode(yb[0]))
    break


Vocabulário: {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, ' ': 5}
Tamanho do vocabulário: 6
Tamanho máximo das sequências (max_len): 5

Exemplo 1:
  Original: deeec
  Codificado: [3, 4, 4, 4, 2]
  Decodificado: deeec
  Reverso (target esperado): ceeed

Exemplo 2:
  Original: cebcc
  Codificado: [2, 4, 1, 2, 2]
  Decodificado: cebcc
  Reverso (target esperado): ccbec

Exemplo 3:
  Original: ceaea
  Codificado: [2, 4, 0, 4, 0]
  Decodificado: ceaea
  Reverso (target esperado): aeaec

Shapes:
  inputs:  torch.Size([50000, 5])
  targets: torch.Size([50000, 5])

Primeiro batch de treino:
  Entradas (xb): torch.Size([128, 5])
  Alvos (yb): torch.Size([128, 5])
  Exemplo de entrada decodificada: eebde
  Exemplo de alvo decodificado: edbee

Excelente — esse output mostra que você **compreendeu e reproduziu um pipeline completo de preparação de dados para Machine Learning**, em um problema de **sequência para sequência (seq2seq)** simples.
Vamos analisar cada parte do resultado com foco em como isso se relaciona com o aprendizado de máquina 👇

---

## 🧩 1️⃣ Vocabulário e codificação

```
Vocabulário: {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, ' ': 5}
Tamanho do vocabulário: 6
```

👉 Isso mostra que você criou um **espaço discreto de símbolos** — uma espécie de *mini universo linguístico*.
Cada caractere foi convertido em um **índice numérico único**, e esse processo é equivalente ao que acontece em NLP (Natural Language Processing), quando as palavras são transformadas em IDs antes de virar embeddings.

**Interpretação ML:**

* Este é o **pré-processamento** de um modelo de linguagem.
* No lugar de palavras, aqui temos letras.
* Esse mapeamento (`char → int`) é a base para permitir que redes neurais operem sobre **números** em vez de texto.

---

## 🧠 2️⃣ Dados de entrada e saída — um problema *seq2seq*

```
Exemplo 1:
  Original: acdba
  Codificado: [0, 2, 3, 1, 0]
  Decodificado: acdba
  Reverso (target esperado): abdca
```

Aqui você definiu um **problema supervisionado** clássico:

* Entrada: uma sequência de letras (`acdba`)
* Saída esperada (target): a **sequência invertida** (`abdca`)

✅ Isso é um *toy problem* (problema de brinquedo) que ajuda a testar se uma rede neural consegue **aprender padrões de sequência**.
Em vez de traduzir entre idiomas (como o Encoder–Decoder faz em tradução), aqui ela precisa **aprender a inverter a sequência** — uma tarefa simples, mas perfeita para estudar aprendizado seqüencial.

---

## 🔢 3️⃣ Estruturas dos tensores

```
inputs:  torch.Size([50000, 5])
targets: torch.Size([50000, 5])
```

Isso quer dizer:

* Temos **50.000 exemplos de treino**.
* Cada exemplo é uma **sequência de 5 tokens**.

📊 Cada linha é um exemplo (amostra) e cada coluna é uma posição da sequência (caractere).
Portanto, o modelo verá isso como uma **matriz de tamanho 50.000 × 5** — um *dataset tabular temporal*.

**Do ponto de vista de ML:**

* Cada linha = uma observação.
* Cada coluna = uma dimensão temporal (ou “posição” no texto).
* Isso está no formato ideal para entrar em uma rede neural do tipo **RNN**, **LSTM** ou **Transformer Encoder–Decoder**.

---

## 🧮 4️⃣ Batch de treinamento

```
Entradas (xb): torch.Size([128, 5])
Alvos (yb): torch.Size([128, 5])
```

O **DataLoader** está dividindo o dataset em *batches* de 128 exemplos.
Isso é essencial para **treinamento eficiente** e **cálculo vetorizado em GPU**.

**Por que isso é importante:**

* As redes neurais aprendem com *gradientes médios por lote*, não amostra a amostra.
* Isso acelera o treino e suaviza o processo de otimização (SGD, Adam, etc.).

---

## 🧩 5️⃣ Conferência de um batch real

```
Exemplo de entrada decodificada: caebd
Exemplo de alvo decodificado: dbeac
```

Aqui você confirmou que:

* A **entrada** é uma sequência aleatória.
* O **alvo** é essa sequência invertida.
  Isso mostra que o dataset está **coerente e limpo**, pronto para o modelo aprender o mapeamento.

---

## 🧠 6️⃣ Interpretação conceitual (visão de ML)

| Etapa                | Conceito ML                        | Analogia                                   |
| -------------------- | ---------------------------------- | ------------------------------------------ |
| Codificação          | Transformar símbolos em números    | Dicionário de tokens                       |
| Padding              | Normalizar tamanho das sequências  | Preencher com “espaço”                     |
| Dataset + Dataloader | Estrutura de treino supervisionado | Como “perguntas e respostas” para o modelo |
| Input/Target         | Aprendizado seq2seq                | Entrada → Saída esperada                   |
| Batch                | Otimização por gradiente           | Treino em mini-grupos                      |

---

## 🎯 Conclusão

✅ **O dataset está bem construído.**
Você implementou, sem usar bibliotecas externas, o pipeline completo que qualquer sistema de NLP moderno (inclusive LLMs) usa em escala — apenas de forma reduzida e didática.

🚀 **Próximo passo natural:**

* Criar um modelo simples (por exemplo, `nn.Embedding + nn.LSTM + nn.Linear`)
* Treiná-lo para aprender a tarefa de reversão (seq2seq)
* Observar se o *loss* diminui e se o modelo aprende a gerar a sequência invertida.

Se quiser, posso gerar esse modelo de rede neural (Encoder–Decoder mínimo em PyTorch) para continuar o experimento. Quer seguir para isso?


## Veja um par

In [ ]:
print(pairs[1])

(tensor([3, 0, 3, 3, 2]), tensor([2, 3, 3, 0, 3]))


# Definição do modelo Seq2Seq com GRU

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_size)
        self.gru = nn.GRU(emb_size, hidden_size, batch_first=True)

    def forward(self, x):
        x = self.embed(x)
        _, h = self.gru(x)
        return h  # [1, B, H]

class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_size)
        self.gru = nn.GRU(emb_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, h):
        """
        x: tensor que indica a parte prévia correta
        h: tensor que indica o estado do encoder da parte prévia
        """
        x = self.embed(x)
        out, h = self.gru(x, h)
        logits = self.fc(out)
        return logits, h # retorna o estado latente para atualizar o estado

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, tgt):
        h = self.encoder(src)
        # usa contexto correto anterior e estado atual para prever o tgt[:, -1]
        logits, _ = self.decoder(tgt[:, :-1], h)
        return logits

# Código para usar o modelo treinado: inferência

---



In [ ]:
def decode_step(decoder, token, h):
    logits, h = decoder(token, h) # obtém logits e atualiza estado da sequência
    next_token = logits[:, -1, :].argmax(-1, keepdim=True)
    return next_token, h

def predict(model, seq, max_len=10):
    model.eval()
    with torch.no_grad():
        src = pad(encode(seq)).unsqueeze(0).to(device, dtype=torch.long)
        h = model.encoder(src) # Obtém estado do modelo após processar entrada inicial

        # 'token' representa a geração passo a passo da sequência invertida
        token = torch.tensor([[vocab[' ']]], dtype=torch.long, device=device)
        seq_invertida = []
        for _ in range(max_len):
            token, h = decode_step(model.decoder, token, h)
            seq_invertida.append(token.item())
        return decode(seq_invertida)

# Preparação para treino

In [ ]:
emb_size = 32
hidden_size = 64
encoder = Encoder(vocab_size, emb_size, hidden_size)
decoder = Decoder(vocab_size, emb_size, hidden_size)
model = Seq2Seq(encoder, decoder).to(device)

loss_fn = nn.CrossEntropyLoss(ignore_index=vocab[' ']) # ignora o pad: " "
opt = torch.optim.Adam(model.parameters(), lr=1e-3)

# Execução do treino

In [ ]:
for epoch in range(20):
    model.train()
    total_loss = 0
    for xb, yb in train_dl:
        xb, yb = xb.to(device, dtype=torch.long), yb.to(device, dtype=torch.long)
        opt.zero_grad()
        logits = model(xb, yb)
        loss = loss_fn(logits.reshape(-1, vocab_size), yb[:, 1:].reshape(-1))
        loss.backward()
        opt.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}: loss={total_loss/len(train_dl):.4f}")

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_size)
        self.gru = nn.GRU(emb_size, hidden_size, batch_first=True)

    def forward(self, x):
        x = self.embed(x)
        _, h = self.gru(x)
        return h  # [1, B, H]

class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_size)
        self.gru = nn.GRU(emb_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, h):
        """
        x: tensor que indica a parte prévia correta
        h: tensor que indica o estado do encoder da parte prévia
        """
        x = self.embed(x)
        out, h = self.gru(x, h)
        logits = self.fc(out)
        return logits, h # retorna o estado latente para atualizar o estado

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, tgt):
        h = self.encoder(src)
        # usa contexto correto anterior e estado atual para prever o tgt[:, -1]
        logits, _ = self.decoder(tgt[:, :-1], h)
        return logits

In [ ]:
emb_size = 32
hidden_size = 64
encoder = Encoder(vocab_size, emb_size, hidden_size)
decoder = Decoder(vocab_size, emb_size, hidden_size)
model = Seq2Seq(encoder, decoder).to(device)

loss_fn = nn.CrossEntropyLoss(ignore_index=vocab[' ']) # ignora o pad: " "
opt = torch.optim.Adam(model.parameters(), lr=1e-3)

print(f"Model initialized and moved to device: {device}")
print(f"Model architecture:\n{model}")

# Vamos testar

In [ ]:
for _ in range(10):
    s = random_seq()
    print(f"Input string: '{s}'") # Added print to show the generated input
    pred = predict(model, s, max_len=len(s))
    print(f"{s} -> {pred}")